# Parte 1
### 1.1 
Elegimos un dataset llamado ["salary.csv"](https://www.kaggle.com/datasets/ayessa/salary-prediction-classification) que tiene de utilidad original crear un algoritmo predicitvo para poder predecir si una persona gana mas de 50.000 USD al año. Creer que existe una desigualdad en la proporcion de quien gana mas dependiendo del sexo en este dataset es valido, es por esta razon que elegimos esta dataset para poder estudiar su posible sesgo.  

Primero, importar los paquetes necesarios para poder utilizar AIF360 y el dataset.

In [23]:
import sys
sys.path.insert(1, "../")

from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing
from IPython.display import Markdown, display
from aif360.datasets import StandardDataset


import pandas as pd
import numpy as np
np.random.seed(0)

salary = pd.read_csv('salary.csv')

## 1.2
Preprocesamos el dataset para poder ocupar las funciones de AIF360.

In [24]:
salary['sex'] = salary['sex'].apply(lambda x: 0 if x==' Female' else 1)
salary['salary'] = salary['salary'].apply(lambda x: 1 if x==' >50K' else 0)
salary_aif = StandardDataset(salary, label_name='salary', protected_attribute_names=['sex'],
                              privileged_classes=[[1]], favorable_classes=[1],
                                features_to_drop=['workclass', 'fnlwgt', 'education', 'marital-status',
                                                  'occupation', 'relationship', 'race', 'native-country'])


salary_aif_train, salary_aif_test = salary_aif.split([0.7], shuffle=True)
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]

In [25]:
print(salary_aif.features.shape, salary_aif_train.features.shape, salary_aif_test.features.shape)

(32561, 6) (22792, 6) (9769, 6)


Ahora el dataset puede ocupar funciones de AIF360, ya que fue procesado correctamente.

## 1.3
Seleccionamos al grupo "Male" como privilegiado, y "Female" como no privilegiado. Decidimos hacer esto ya que el numero de articulos que apoya esta suposicion es bien grande. [Por ejemplo](https://www.payscale.com/gender-lifetime-earnings-gap) en este articulo establecen que los hombres ganan mas que las mujeres, pero no *tanto* mas cuando trabajan el mismo oficio.

## 1.4
Vamos a calcular dos metricas. La diferencia en el promedio, en donde se resta los resultados favorables para el grupo privilegiado con el grupo no privilegiado. Tambien vamos a calcular "Disparate Impact" para poder observar la proporcion de resultados favorables para el grupo no privilegiado en comparacion del grupo privilegiado.

In [27]:
mean_train = BinaryLabelDatasetMetric(salary_aif_train,
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
print("Diferencia en promedio = %f" % mean_train.mean_difference())
print("Disparate Impact = %f" % mean_train.disparate_impact())

Diferencia en promedio = -0.195396
Disparate Impact = 0.363245


Como el resultado de la diferencia en promedio (mean_difference) es negativo, indica que hay menos resultados favorables para el grupo no privilegiado, por lo que si existe un sesgo.